# Grupper patienter med unsupervised clustering med k-means

## Hent data fra kaggle og put det i dataframes

https://www.kaggle.com/competitions/k-means-clustering-for-heart-disease-analysis/data

In [1]:
import pandas as pd

heart_disease_csv = pd.read_csv('data/heart_disease.csv')
sample_csv = pd.read_csv('data/sample.csv')

heart_disease_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
dtypes: float64(5), int64(2), object(8)
memory usage: 107.9+ KB


## Filtrer dataen så jeg kun bruger patienter som har et id fra sample filen. (Det var en del af opgaven)

In [2]:
valid_ids = set(sample_csv['id'])
data = heart_disease_csv[heart_disease_csv['id'].isin(valid_ids)]

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 299 entries, 0 to 919
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        299 non-null    int64  
 1   age       299 non-null    int64  
 2   sex       299 non-null    object 
 3   dataset   299 non-null    object 
 4   cp        299 non-null    object 
 5   trestbps  278 non-null    float64
 6   chol      292 non-null    float64
 7   fbs       277 non-null    object 
 8   restecg   298 non-null    object 
 9   thalch    280 non-null    float64
 10  exang     280 non-null    object 
 11  oldpeak   279 non-null    float64
 12  slope     196 non-null    object 
 13  ca        113 non-null    float64
 14  thal      144 non-null    object 
dtypes: float64(5), int64(2), object(8)
memory usage: 37.4+ KB


## Ryd op i data

fjern alle rows med nan data, da det vil ødelægge vores resultater. Jeg har tænkt at der måske er bedre måder at gøre det på. måske at bare fjerne den colonne med mange nan, eller på en måde fylde det ud med mock data af samme statiske frekvens som fyldt data, Dette kaldes for Mean Imputation. Men jeg føler der er for mange faktorer i dataen til at jeg kan gøre det, så som hvis de data man fylder ind har en direkte relation til noget andet data, og det ikke bliver reflekteret i den tilfældigt indsatte data. Da vi har med medicinsk data, tænker jeg at det er yderst vigtigt at data'en er korrekt. At indsætte noget tilfældig data ville ikke være hensigtsmæssigt i denne sammenhæng, også selv om vi får betydeligt færre samples.

dropper også id da den ikke har betydning for dataen.

### Gammel måde hvor jeg droppede alle subjects med manglende værdier:

In [3]:
#GAMMEL MÅDE
#

#data = data.dropna()
#data = data.drop('id', axis=1)

#data.to_csv('data/heart_disease_filtered.csv', index=False)

#print(data['age'].count())

Jeg har kørt det hele med alt null data droppet.
Men jeg kan vurdere at der ikke er nok data til at træne og teste modellen.

Jeg bliver nødt til at indsætte data så godt jeg kan, for at få et dybere datasæt.

### Ny måde hvor jeg udskifter alle manglende værdier
med en default værdi af 0 for tal og "Not known" for strings. det vil svare til at lave en ny kategori som hedder "Not known" og 0 i forhold til vores clustering. Det er bedre end at indsætte tilfældige værdier, fordi det ville forvringe resultatet. Det kunne også hjælpe lægerne på en eller anden måde tænker jeg, at have en kategori for sig selv til dem de ikke har data nok på.

Der er muligvis et problem i at lave Boolean manglende værdi om til "not known". som jeg normalisere dataen kommer det til at blive 0,1,2 kategorier for true of false værdier. Men jeg kan ikke forstille mig at bruge tilfældige værdier i stedet for... men måske det er bedre.

In [4]:
#NY MÅDE

data = data.drop('id', axis=1)
data.isna().sum()

age           0
sex           0
dataset       0
cp            0
trestbps     21
chol          7
fbs          22
restecg       1
thalch       19
exang        19
oldpeak      20
slope       103
ca          186
thal        155
dtype: int64

In [5]:
for col in data:
    if data[col].dtype == 'object':
        data[col].fillna('Not known', inplace=True)
    if data[col].dtype == 'int64':
        data[col].fillna(data[col].mode()[0],inplace=True)
    if data[col].dtype == 'float64':
        data[col].fillna(data[col].mean(),inplace=True)
        
data.isna().sum()

age         0
sex         0
dataset     0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalch      0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
dtype: int64

In [6]:
data

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
0,63,Male,Cleveland,typical angina,145.000000,233.0,True,lv hypertrophy,150.000000,False,2.300000,downsloping,0.000000,fixed defect
1,67,Male,Cleveland,asymptomatic,160.000000,286.0,False,lv hypertrophy,108.000000,True,1.500000,flat,3.000000,normal
3,37,Male,Cleveland,non-anginal,130.000000,250.0,False,normal,187.000000,False,3.500000,downsloping,0.000000,normal
4,41,Female,Cleveland,atypical angina,130.000000,204.0,False,lv hypertrophy,172.000000,False,1.400000,upsloping,0.000000,normal
7,57,Female,Cleveland,asymptomatic,120.000000,354.0,False,normal,163.000000,True,0.600000,upsloping,0.000000,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,51,Female,VA Long Beach,asymptomatic,114.000000,258.0,True,lv hypertrophy,96.000000,False,1.000000,upsloping,0.716814,Not known
911,62,Male,VA Long Beach,asymptomatic,160.000000,254.0,True,st-t abnormality,108.000000,True,3.000000,flat,0.716814,Not known
912,53,Male,VA Long Beach,asymptomatic,144.000000,300.0,True,st-t abnormality,128.000000,True,1.500000,flat,0.716814,Not known
916,62,Male,VA Long Beach,typical angina,131.539568,139.0,False,st-t abnormality,140.014286,Not known,0.869892,Not known,0.716814,Not known


# Gem i ny csv fil

In [7]:
data.to_csv('data/heart_disease_cleaned.csv', index=False)